In [ ]:
from __future__ import print_function
import os, sys
#module_path = os.path.abspath(os.path.join('..'))
#if module_path not in sys.path:
#    sys.path.append(module_path)
import KerasTools as KT
import ipywidgets
import StringIO
import matplotlib.pyplot as plt
import skimage
import time
import keras

In [ ]:
button_ly = ipywidgets.Layout(width='34px', height='30px', border='1px solid black')

bleft_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
bleft = ipywidgets.Button(layout=button_ly, style=bleft_st, icon='arrow-left')
bright_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
bright = ipywidgets.Button(layout=button_ly, style=bright_st, icon='arrow-right')
bstay_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
bstay = ipywidgets.Button(layout=button_ly, style=bstay_st, icon='dot-circle-o')
brotleft_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
brotleft = ipywidgets.Button(layout=button_ly, style=brotleft_st, icon='rotate-left')
brotright_st = ipywidgets.ButtonStyle(font_weight='bold', button_color='yellow')
brotright = ipywidgets.Button(layout=button_ly, style=brotright_st, icon='rotate-right')

lt = ipywidgets.Layout(width='68px', height='30px', border='1px solid black')
stt = ipywidgets.ButtonStyle(font_weight='bold', button_color='green')
bnew = ipywidgets.Button(layout=lt, style=stt, description = 'New')
stat = ipywidgets.HTML(layout=lt)

ls = ipywidgets.Layout(width='34px', height='30px', border=None)
bs = ipywidgets.Label(layout=ls)
    
bx = ipywidgets.HBox([bnew, bleft, brotleft, bstay, brotright, bright, stat])
img = ipywidgets.Image()
game = KT.qlearn.tromis.Tromis(width=7,height=12,max_turn=None)

total_score = 0

def plot_game(frame):
    imbuf = StringIO.StringIO()
    f = skimage.transform.resize(frame,(12*256/7,256), order=0, mode='constant', anti_aliasing=False)
    plt.imsave(imbuf, f)
    imbuf.seek(0)           
    img.value = imbuf.buf

def onButtonClicked(args):
    global total_score
    if args.description == 'New':
        game.reset()
        stat.value = ""
        total_score = 0
    elif not game.is_over():
        args.style.button_color = 'red'
        time.sleep(0.1)
        args.style.button_color = 'yellow'        
        action = {'arrow-left': 0, 'arrow-right': 1, 'dot-circle-o': 2, 'rotate-left': 3, 'rotate-right': 4}
        game.play(action[args.icon])
    current_reward = game.get_score()
    total_score += current_reward
    stat.value = "<center><big><b>{}/{}".format(current_reward, total_score)
    
    plot_game(game.get_frame())

bleft.on_click(onButtonClicked)
bright.on_click(onButtonClicked)
bstay.on_click(onButtonClicked)
bnew.on_click(onButtonClicked)
brotleft.on_click(onButtonClicked)
brotright.on_click(onButtonClicked)


display(img)
display(bx)
plot_game(game.get_frame())

In [ ]:
model = keras.models.load_model('tromis_7x12_130.h5')
model.summary()
game = KT.qlearn.tromis.Tromis(width=7,height=12,max_turn=None)
agent = KT.qlearn.agent.Agent(model=model, nb_frames = 12)

In [ ]:
button_map = {0: bleft, 1: bright, 2: bstay, 3:brotleft, 4:brotright}
num_games = 10
won = 0
for i in range(num_games):
    while not game.is_over():
        time.sleep(0.1)
        action = agent.get_action(game)
        button_map[action].click()
    time.sleep(1)
    if game.is_won(): won += 1
    bnew.click()
print("Won {:2.0%} of games".format(float(won)/num_games))